# Parametrized Fully Connected Network in Fugu

This notebook builds a fully connected network (input → hidden → output) using Fugu bricks and scaffold, runs it on the snn_Backend for 20 steps, and verifies connectivity.

1. A fully connected network : 2x2. One input layer, 2 hidden layers, output layer.

Dummy inputs

In [1]:
import numpy as np
from fugu import Scaffold
from fugu.bricks.input_bricks import Vector_Input
from fugu.bricks.dense_bricks import dense_layer_1d
from fugu.backends import snn_Backend

# Parametrize network dimensions

input_size = 2   
output_size = 2
n_steps = 20

# Create dummy input spikes 
spikes = np.ones((input_size, n_steps))


# Build scaffold and add input brick 
scaffold = Scaffold()
scaffold.add_brick(
    Vector_Input(spikes, coding='Raster', name='Input', time_dimension=True),
    input_nodes='input'
 )

# Add output layer brick (fully connected to input) 
W = np.ones((output_size, input_size)) # shape (2, 2)
T = np.ones(output_size) * 0.5
                # shape (2,)
scaffold.add_brick(
    dense_layer_1d(output_shape=(output_size,), weights=W, thresholds=T, name='hidden0'),
    input_nodes=[-1],
    output=False
)
scaffold.add_brick(
    dense_layer_1d(output_shape=(output_size,), weights=W, thresholds=T, name='hidden1'),
    input_nodes=[-1],
    output=False
)
scaffold.add_brick(
    dense_layer_1d(output_shape=(output_size,), weights=W, thresholds=T, name='Output'),
    input_nodes=[-1],
    output=True
)

print("Weights shape:", W.shape)
print("Thresholds shape:", T.shape)

 #Build the network graph 
scaffold.lay_bricks()

# print all edges 
print('Network edges (source → target):')
for u, v, d in scaffold.graph.edges(data=True):
    print(f'{u} → {v} | weight={d.get("weight")} | delay={d.get("delay")}')

#  Run simulation 
backend = snn_Backend()
# backend = loihi_backend()
backend.compile(scaffold, compile_args={'record': 'all', 'debug_mode': False})
result = backend.run(n_steps=n_steps)

# Show result summary 
print('Simulation result (spike times):')
print(result)

Weights shape: (2, 2)
Thresholds shape: (2,)
Network edges (source → target):
Input-0:begin → Input-0:complete | weight=1.0 | delay=19
Input-0:begin → hidden0_begin | weight=0.0 | delay=1
Input-0:complete → hidden0_complete | weight=0.0 | delay=1
Input-0:(0,) → hidden0d0 | weight=1.0 | delay=1
Input-0:(0,) → hidden0d1 | weight=1.0 | delay=1
Input-0:(1,) → hidden0d0 | weight=1.0 | delay=1
Input-0:(1,) → hidden0d1 | weight=1.0 | delay=1
hidden0_begin → hidden1_begin | weight=0.0 | delay=1
hidden0_complete → hidden1_complete | weight=0.0 | delay=1
hidden0d0 → hidden1d0 | weight=1.0 | delay=1
hidden0d0 → hidden1d1 | weight=1.0 | delay=1
hidden0d1 → hidden1d0 | weight=1.0 | delay=1
hidden0d1 → hidden1d1 | weight=1.0 | delay=1
hidden1_begin → Output_begin | weight=0.0 | delay=1
hidden1_complete → Output_complete | weight=0.0 | delay=1
hidden1d0 → Outputd0 | weight=1.0 | delay=1
hidden1d0 → Outputd1 | weight=1.0 | delay=1
hidden1d1 → Outputd0 | weight=1.0 | delay=1
hidden1d1 → Outputd1 | weig

2. Fixed bug. Fully connected layers no longer have to be of the same size to be fully connected to one another

In [ ]:
import numpy as np
from fugu import Scaffold
from fugu.bricks.input_bricks import Vector_Input
from fugu.bricks.dense_bricks import dense_layer_1d
from fugu.backends import snn_Backend

# Parametrize network dimensions

input_size = 5   
output_size = 2
n_steps = 100

# Create dummy input spikes 
spikes = np.ones((input_size, n_steps))
print("Input spikes shape:", spikes)


# Build scaffold and add input brick 
scaffold = Scaffold()
scaffold.add_brick(
    Vector_Input(
        spikes, name='Input', time_dimension=True),
    input_nodes='input'
 )

# Add output layer brick (fully connected to input) 
W = np.ones((output_size, input_size))   # shape (2, 4)
T = np.ones(output_size) * 0.25              # shape (2,)
scaffold.add_brick(
    dense_layer_1d(output_shape=(output_size,), weights=W, thresholds=T, name='Output'),
    input_nodes=[-1],
    output=True
)


print("Weights shape:", W.shape)
print("Thresholds shape:", T.shape)

 #Build the network graph 
scaffold.lay_bricks()

# print all edges 
print('Network edges (source → target):')
for u, v, d in scaffold.graph.edges(data=True):
    print(f'{u} → {v} | weight={d.get("weight")} | delay={d.get("delay")}')

#  Run simulation 
backend = snn_Backend()
# backend = loihi_backend()
backend.compile(scaffold, compile_args={'record': 'all', 'debug_mode': False})
result = backend.run(n_steps=n_steps)

print('Simulation result (spike times):')
print(result)

Input spikes shape: [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

3. Test network. 5x3x2x4

In [1]:
import numpy as np
from fugu import Scaffold
from fugu.bricks.input_bricks import Vector_Input
from fugu.bricks.dense_bricks import dense_layer_1d
from fugu.backends import snn_Backend

# Parametrize network dimensions
input_size = 5
hidden_size = 3
mid_size = 2
output_size = 4
n_steps = 10

# Create sparse, well-timed input spikes (each neuron spikes once at a unique time)
spikes = np.zeros((input_size, n_steps))
for i in range(input_size):
    spikes[i, i * (n_steps // input_size)] = 1  # Each neuron spikes once, spaced out

# Build scaffold and add input brick 
scaffold = Scaffold()
A = scaffold.add_brick(
    Vector_Input(spikes=spikes, coding='Raster', name='Input', time_dimension=True),
    input_nodes='input'
 )

# Add hidden layer brick (5 -> 3) 
W1 = np.random.uniform(0.2, 1.0, (hidden_size, input_size))   # shape (3, 5)
T1 = np.random.uniform(0.2, 1.0, hidden_size)                 # shape (3,)
B = scaffold.add_brick(
    dense_layer_1d(output_shape=(hidden_size,), weights=W1, thresholds=T1, name='Hidden'),
    input_nodes=[-1],
    output=False
 )

#  Add mid layer brick (3 -> 2)
W2 = np.random.uniform(0.2, 1.0, (mid_size, hidden_size))     # shape (2, 3)
T2 = np.random.uniform(0.2, 1.0, mid_size)                   # shape (2,)
C = scaffold.add_brick(
    dense_layer_1d(output_shape=(mid_size,), weights=W2, thresholds=T2, name='Mid'),
    input_nodes=[-1],
    output=False
 )

# Add output layer brick (2 -> 4) 
W3 = np.random.uniform(0.2, 1.0, (output_size, mid_size))     # shape (4, 2)
T3 = np.random.uniform(0.2, 1.0, output_size)                # shape (4,)
D = scaffold.add_brick(
    dense_layer_1d(output_shape=(output_size,), weights=W3, thresholds=T3, name='output'),
    input_nodes=[-1],
    output=True
 )

# Build the network graph
scaffold.lay_bricks()

#  print all edges
print('Network edges (source → target):')
for u, v, d in scaffold.graph.edges(data=True):
    print(f'{u} → {v} | weight={d.get("weight")} | delay={d.get("delay")}')

backend = snn_Backend()
backend.compile(scaffold, compile_args={'record': 'all', 'debug_mode': True})
result = backend.run(n_steps=n_steps)

print('Simulation result (spike times):')
print(result)

Network edges (source → target):
Input-0:begin → Input-0:complete | weight=1.0 | delay=9
Input-0:begin → Hidden_begin | weight=0.0 | delay=1
Input-0:complete → Hidden_complete | weight=0.0 | delay=1
Input-0:(0,) → Hiddend0 | weight=0.5014518206824022 | delay=1
Input-0:(0,) → Hiddend1 | weight=0.7933482629231565 | delay=1
Input-0:(0,) → Hiddend2 | weight=0.6495824881539075 | delay=1
Input-0:(1,) → Hiddend0 | weight=0.7735735010460296 | delay=1
Input-0:(1,) → Hiddend1 | weight=0.8862123561108776 | delay=1
Input-0:(1,) → Hiddend2 | weight=0.43190090387457086 | delay=1
Input-0:(2,) → Hiddend0 | weight=0.5059904966627514 | delay=1
Input-0:(2,) → Hiddend1 | weight=0.4204057468421527 | delay=1
Input-0:(2,) → Hiddend2 | weight=0.3897606461895995 | delay=1
Input-0:(3,) → Hiddend0 | weight=0.9552933842394389 | delay=1
Input-0:(3,) → Hiddend1 | weight=0.5549115159625716 | delay=1
Input-0:(3,) → Hiddend2 | weight=0.25745771509092824 | delay=1
Input-0:(4,) → Hiddend0 | weight=0.6907986798064008 | d